<a href="https://colab.research.google.com/github/juanmijael-salazar/my_repository/blob/master/programacion%20con%20CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Trabajando con cuda y gpu en google

In [1]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dkg -l | grep cuda- | awk '{print $2}' | xargs -nl dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-fs-prebuilt' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'


In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update 
!apt-get install cuda

--2021-12-03 22:11:12--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-12-03 22:11:12 ERROR 404: Not Found.

dpkg-deb: error: 'cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb' is not a Debian format archive
dpkg: error processing archive cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb (--install):
 dpkg-deb --control subprocess returned error exit status 2
Errors were encountered while processing:
 cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
gpg: can't open '/var/cuda-repo-9-2-local/7fa2af80.pub': No such file or directory
Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.down

#version cuda

In [2]:
!nvcc --version

/bin/bash: nvcc: command not found


#instalamos extencion para ejecutar nvcc

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-6jh7k34i
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-6jh7k34i
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=02aaecdaa1ba4ea19f23080f477d2628565e1f340aa9f6ac22613bd58a2ce6bb
  Stored in directory: /tmp/pip-ephem-wheel-cache-msg7z1_s/wheels/c5/2b/c0/87008e795a14bbcdfc7c846a00d06981916331eb980b6c8bdf
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include<stdio.h>
#include<cuda.h>

int main()
{
    cudaDeviceProp p;
    int device_id;
    int major;
    int minor;
    int warp;

    cudaGetDevice(&device_id);
    cudaGetDeviceProperties(&p,device_id);

    major=p.major;
    minor=p.minor;
    warp=p.warpSize;

    printf("El nombre de la GPU en su sistema es %s\n",p.name);

    printf("\n La capacidad de computo de una GPU actual en su sistema es %d.%d",major,minor);

    printf("\n La capacidad de warps de una GPU actual en su sistema es %d",p.warpSize);

    return 0;
}

El nombre de la GPU en su sistema es Tesla K80

 La capacidad de computo de una GPU actual en su sistema es 3.7
 La capacidad de warps de una GPU actual en su sistema es 32


In [ ]:
%%cu
#include<stdio.h>
#include<cuda.h>

__global__ void arradd(int *x,int *y,int *z)  //kernel definition
{
  int id=blockIdx.x;
 /* blockIdx.x gives the respective block id wich starts from 0 */
  z[id]=x[id]+y[id];
}

int main()
{
    int a[6]={2,5,9,4,8,7};
    int b[6]={4,7,10,1,6,3};
    int c[6];
    int *d,*e,*f;
    int i;

    cudaMalloc((void **)&d,6*sizeof(int));
    cudaMalloc((void **)&e,6*sizeof(int));
    cudaMalloc((void **)&f,6*sizeof(int));
 
    cudaMemcpy(d,a,6*sizeof(int),cudaMemcpyHostToDevice);
    cudaMemcpy(e,b,6*sizeof(int),cudaMemcpyHostToDevice);

/**
    for(i=0;i<6;i++)
    {
        c[i]= a[i] + b[i];
    }
**/

    arradd<<<6,1>>>(d,e,f);
 
    cudaMemcpy(c,f,6*sizeof(int),cudaMemcpyDeviceToHost);


    printf("\nSum of two arrays:\n ");
    for(i=0;i<6;i++)
    {
        printf("%d\t",c[i]);
    }

    cudaFree(d);
    cudaFree(e);
    cudaFree(f);
 
    return 0;
}


Sum of two arrays:
 0	0	0	0	0	0	
